In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from scipy.stats import pearsonr
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
from scipy.interpolate import interp1d

from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

from numba import jit, float64, int64

In [2]:
@jit(nopython=True)
def f(x, t, z, tau, delta, sigma=1):
    term1 = 1/np.sqrt(2 * np.pi * sigma**2 * (t-tau))
    term2 = -(x - z - delta * (t-tau))**2 / (2 * sigma**2 * (t-tau))
    return term1 * np.exp(term2)

@jit(nopython=True)
def psi(threshold, lamda, t, z, tau, delta, sigma=1):
    db = -lamda * threshold * np.exp(-lamda*t)
    term1 = 0.5*f(threshold * np.exp(-lamda*t), t, z, tau, delta, sigma)
    term2 = db - delta - (threshold * np.exp(-lamda*t) - z - delta * (t-tau))/(t-tau)
    return term1 * term2

@jit(nopython=True)
def fpt(threshold, lamda, delta, z=0, sigma=1, dt=0.02, T_max=5):
    gu = np.zeros((int(T_max/dt)+2,))
    gl = np.zeros((int(T_max/dt)+2,))
    T = np.zeros((int(T_max/dt)+2,))
    
    gu[1] = -2*psi(threshold, lamda, dt, z, 0, delta, sigma)
    gl[1] =  2*psi(-threshold, lamda, dt, z, 0, delta, sigma)
    T[1] = dt
    
    for n in range(2, int(T_max/dt)+2):
        su = -2 * psi( threshold, lamda, n*dt, z, 0, delta, sigma)
        sl =  2 * psi(-threshold, lamda, n*dt, z, 0, delta, sigma)
        
        for j in range(1, n):
            if threshold * np.exp(-lamda*j*dt) == 0:
                continue
            
            psi_n_j_pp = psi( threshold, lamda, n*dt,  threshold * np.exp(-lamda*j*dt), j*dt, delta, sigma)
            psi_n_j_pn = psi( threshold, lamda, n*dt, -threshold * np.exp(-lamda*j*dt), j*dt, delta, sigma)
            psi_n_j_np = psi(-threshold, lamda, n*dt,  threshold * np.exp(-lamda*j*dt), j*dt, delta, sigma)
            psi_n_j_nn = psi(-threshold, lamda, n*dt, -threshold * np.exp(-lamda*j*dt), j*dt, delta, sigma)
            
            su +=  2 * dt * (gu[j] * psi_n_j_pp + gl[j] * psi_n_j_pn)
            sl += -2 * dt * (gu[j] * psi_n_j_np + gl[j] * psi_n_j_nn)
            
        gu[n] = su
        gl[n] = sl
        T[n] = (n*dt)
    return gu, gl, T

In [3]:
def CDDM_likelihood(prms, RT, Z):
    ub = lambda t: prms[0] * np.exp(-prms[1]*t)
    lb = lambda t: -1*ub(t)
    dub = lambda t: -prms[1] * prms[0] * np.exp(-prms[1]*t)
    dlb = lambda t: -1*dub(t)

    delta = prms[2]
    t0 = prms[3]
    sig = prms[4]
    
    T_max = np.max(np.abs(RT))
    gu, gl, TT = fpt(prms[0], prms[1], 0, z=0, dt=0.02, T_max=T_max)
    
    gtup = interp1d(TT, gu)
    gtlp = interp1d(TT, gl)
    
    ll = 0
    for i in range(len(RT)):
        if np.abs(RT[i])-t0 > 0:
            
            ll += 0.5*(np.log(Z[i]) - np.log(t0) + 0.5*sig**2)**2/sig**2 + 0.5*np.log(2*np.pi*sig**2*Z[i]**2)
            
            if RT[i]>=0:
                exp_term = np.exp(delta*ub(np.abs(RT[i])-t0) - 0.5*delta**2*(np.abs(RT[i])-t0))
                density = exp_term*gtup(np.abs(RT[i])-t0)
            else:                
                exp_term = np.exp(delta*lb(np.abs(RT[i])-t0) - 0.5*delta**2*(np.abs(RT[i])-t0))
                density = exp_term*gtlp(np.abs(RT[i])-t0)
                
            if density>1e-14:
                ll += -np.log(density)
            else:
                ll += -np.log(1e-14) 
        else:
            ll += -np.log(1e-14)
    
    return ll

In [4]:
data = pd.read_csv('../../_Data/Study2.csv', 
                   index_col=0).reset_index(drop=True)

data = data.sort_values(by=['participant', 'trials', 'event'])

# # data = data[data.cue == 'SP']
# data = data[data.cue == 'AC']
# data = data[data.event == 3] # This event corresponds to decision time and the rest correspond to non-decision time
data = data[data['Duration']<data['RT']].reset_index(drop=True)

data['RT']/=1000
data['Duration']/=1000

In [5]:
data['participant'].nunique()

25

In [6]:
data

,participant,trials,event,component,Duration,samples,stim,resp,RT,cue,movement,trigger,data,correct
0,processed_500Hz_0001_epo,2,4,0,0.472,0,1.0,resp_right,1.068,AC,stim_left,AC/stim_left/resp_right,4.967953e-06,0
1,processed_500Hz_0001_epo,4,4,0,0.862,0,1.0,resp_left,1.352,AC,stim_left,AC/stim_left/resp_left,-6.663091e-06,1
2,processed_500Hz_0001_epo,6,4,0,0.186,0,2.0,resp_right,0.724,AC,stim_right,AC/stim_right/resp_right,-1.658713e-06,1
3,processed_500Hz_0001_epo,7,4,0,0.454,0,2.0,resp_right,1.214,AC,stim_right,AC/stim_right/resp_right,-3.872259e-06,1
4,processed_500Hz_0001_epo,11,4,0,0.462,0,1.0,resp_right,0.753,AC,stim_left,AC/stim_left/resp_right,-2.844621e-06,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2319,processed_500Hz_0025_epo,185,4,0,0.342,0,1.0,resp_left,0.828,AC,stim_left,AC/stim_left/resp_left,7.214000e-06,1
2320,processed_500Hz_0025_epo,187,4,0,0.120,0,1.0,resp_left,0.553,AC,stim_left,AC/stim_left/resp_left,-2.039230e-06,1
2321,processed_500Hz_0025_epo,192,4,0,0.652,0,2.0,resp_left,1.440,AC,stim_right,AC/stim_right/resp_left,9.874027e-07,0
2322,processed_500Hz_0025_epo,196,4,0,0.506,0,2.0,resp_right,0.837,AC,stim_right,AC/stim_right/resp_right,1.403884e-05,1


In [7]:
prms_dc = {'sbj':[],
           'b0':[],
           'lambda':[],
           'delta':[],
           't0':[],
           'sigma':[],
           'mean_z':[],
           'std_z':[],
           'G2':[],
           'BIC':[]}

min_b0 = 0.5
max_b0 = 4

min_lambda = .01
max_lambda = 4

min_ndt = 0.05
max_ndt = 1

min_delta = -5
max_delta = 5

min_sig = 0.01
max_sig = 2

bounds = [(min_b0, max_b0), (min_lambda, max_lambda), 
          (min_delta, max_delta), (min_ndt, max_ndt), 
          (min_sig, max_sig)]

In [8]:
for sbj in tqdm(data.participant.unique()):
    sbj_data = data[data['participant']==sbj]
    choice = 2*sbj_data.correct.values.astype(np.int64)-1
    RT = choice*sbj_data.RT.values
    Z = sbj_data.RT.values-sbj_data.Duration.values

    min_ans = differential_evolution(CDDM_likelihood,
                                         args=(RT, Z),
                                         bounds= bounds)

    min_ans = minimize(CDDM_likelihood,
                       args=(RT, Z),
                       method='nelder-mead',
                       x0=min_ans.x,
                       bounds=bounds)
    prms_dc['sbj'].append(sbj)
    prms_dc['b0'].append(min_ans.x[0])
    prms_dc['lambda'].append(min_ans.x[1])
    prms_dc['delta'].append(min_ans.x[2])
    prms_dc['t0'].append(min_ans.x[3])
    prms_dc['sigma'].append(min_ans.x[4])
    prms_dc['mean_z'].append(np.mean(Z))
    prms_dc['std_z'].append(np.std(Z))
    prms_dc['G2'].append(2*min_ans.fun)
    prms_dc['BIC'].append(2*min_ans.fun + 5 * np.log(RT.shape[0]))

100%|███████████████████████████████████████████| 25/25 [01:23<00:00,  3.36s/it]


In [9]:
prms_df = pd.DataFrame(prms_dc)

# prms_df.to_csv('_prms/exp_speed.csv', index=False)
prms_df.to_csv('_prms/exp_accuracy.csv', index=False)